In [1]:
import sys
sys.path.append('..')
import doctable
import numpy as np

In [2]:
def make_payload(sizeGB, num):
    # 5000000000 (5e9) is 45 GB max, ~38GB stable.
    siz = int(1e8 * sizeGB)
    payload = [(i,np.ones(siz)) for i in range(num)]
    return payload

In [3]:
def insert_test(db, payload):
    for i,p in payload:
        db.insert({'id':i, 'payload':p})

def select_test(db):
    n = len(db.select('payload'))
    #db.delete() # empty database
    return n

In [4]:
# define and instantiate databases
class FileDB(doctable.DocTable):
    __tabname__  = 'filetable'
    __args__ = {'new_db':True}
    __schema__ = (
        ('idcol', 'id'),
        ('picklefile', 'payload'),
    )

class BinDB(doctable.DocTable):
    __tabname__  = 'filetable'
    __args__ = {'new_db':True}
    __schema__ = (
        ('idcol', 'id'),
        ('pickle', 'payload'),
    )
fdb, bdb = FileDB(target='filedbtest.db'), BinDB(target='bindbtest.db')
fdb.delete(), bdb.delete() # empty datbases
print(fdb, bdb)

<DocTable::sqlite:///filedbtest.db:filetable ct: 0> <DocTable::sqlite:///bindbtest.db:filetable ct: 0>


In [5]:
fdb.delete()

In [6]:
# payload and run tests
p = make_payload(1, 20)
%time insert_test(bdb, p)
del p
%time select_test(bdb)
bdb.delete()

CPU times: user 26.5 s, sys: 33.1 s, total: 59.6 s
Wall time: 1min 48s
CPU times: user 12.6 s, sys: 19.8 s, total: 32.4 s
Wall time: 32.4 s


In [8]:
sett = [(1, 5), (1, 10), (1, 20), (2, 5), (0.5, 50), (0.1,100)]

for siz, num in sett[::-1]:
    print(f'____{num} chunks of {siz} GB ____')
    p = make_payload(siz, num)
    
    %time insert_test(bdb, p)
    del p
    
    %time select_test(bdb)
    bdb.delete()
    
    print('\n\n')

____100 chunks of 0.1 GB ____
CPU times: user 15.7 s, sys: 28.9 s, total: 44.6 s
Wall time: 46.5 s
CPU times: user 6 s, sys: 8.42 s, total: 14.4 s
Wall time: 14.4 s



____50 chunks of 0.5 GB ____
CPU times: user 34.8 s, sys: 1min 3s, total: 1min 37s
Wall time: 2min 31s
CPU times: user 15.7 s, sys: 25 s, total: 40.7 s
Wall time: 40.7 s



____5 chunks of 2 GB ____


InterfaceError: (sqlite3.InterfaceError) Error binding parameter 1 - probably unsupported type.
[SQL: INSERT OR FAIL INTO filetable (id, payload) VALUES (?, ?)]
[parameters: (0, <memory at 0x7f484eb8e120>)]
(Background on this error at: http://sqlalche.me/e/rvf5)